# **LaTeX table with the BAO-fit results**

This notebook shows how to create a LaTeX table with the BAO-fit results from a set of mocks. It also makes some plots

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
import pandas as pd
from utils_data import GetThetaLimits
from utils_baofit import BAOFitInitializer

# # Option A. DESY6-related stuff
# dataset_mocks_list = ["DESY6_COLA"]
# # dataset_mocks_list = ["DESY6_COLA_DR1tiles_noDESI"]
# # dataset_mocks_list = ["DESY6_COLA_DR1tiles_DESIonly"]
# # dataset_mocks_list = ["DESY6_COLA_dec_below-23.5"]
# # dataset_mocks_list = ["DESY6_COLA_dec_above-23.5"]
# dataset_mocks_list = ["DESY6_COLA", "DESY6_COLA_DR1tiles_noDESI", "DESY6_COLA_DR1tiles_DESIonly", "DESY6_COLA_dec_below-23.5", "DESY6_COLA_dec_above-23.5"]
# dynamical_theta_limits = False
# old_template = "_old"
# # old_template = ""
# delta_z = None # only for DESI stuff
# pow_broadband = [-2, -1, 0]


# Option B. DESI-related stuff
delta_z = 0.02
dataset_mocks_list = [
    f"DESIY1_LRG_Abacus_altmtl_deltaz{delta_z}",
    f"DESIY1_LRG_Abacus_altmtl_deltaz{delta_z}_LRG1",
    f"DESIY1_LRG_Abacus_altmtl_deltaz{delta_z}_LRG2",
    f"DESIY1_LRG_Abacus_altmtl_deltaz{delta_z}_LRG3"
]
dynamical_theta_limits = True
pow_broadband = [-1, 0]
# pow_broadband = [-2, -1, 0]
# pow_broadband = [-2, -1, 0, 1]
# pow_broadband = [-3, -2, -1, 0, 1]


fit_results = {}
mock_id_detected_list = {}

for dataset in dataset_mocks_list:

    dataset_orig = dataset
    bins_removed = []

    if delta_z is not None:
        if delta_z == 0.028:
            if "DESIY1" in dataset:
                if "LRG1" in dataset:
                    bins_removed = list(map(int, np.arange(7, 25)))  # LRG1
                    dataset_orig = dataset.replace("_LRG1", "")
                elif "LRG2" in dataset:
                    bins_removed = list(map(int, np.concatenate([np.arange(0, 7), np.arange(14, 25)])))  # LRG2
                    dataset_orig = dataset.replace("_LRG2", "")
                elif "LRG3" in dataset:
                    bins_removed = list(map(int, np.arange(0, 14)))  # LRG3
                    dataset_orig = dataset.replace("_LRG3", "")
        elif delta_z == 0.02:
            if "DESIY1" in dataset:
                if "LRG1" in dataset:
                    bins_removed = list(map(int, np.arange(10, 35)))  # LRG1
                    dataset_orig = dataset.replace("_LRG1", "")
                elif "LRG2" in dataset:
                    bins_removed = list(map(int, np.concatenate([np.arange(0, 10), np.arange(20, 35)])))  # LRG2
                    dataset_orig = dataset.replace("_LRG2", "")
                elif "LRG3" in dataset:
                    bins_removed = list(map(int, np.arange(0, 20)))  # LRG3
                    dataset_orig = dataset.replace("_LRG3", "")

    print(f"Loading the results for the {dataset} data set...")

    if "DESIY1" in dataset:
        nz_flag = "mocks"
        cov_type = "mocks"
        cosmology_template = "desifid"
        cosmology_covariance = "desifid"
    
        if "EZ" in dataset:
            n_mocks = 1000
        elif "Abacus" in dataset:
            n_mocks = 25
        
    elif "DESY6" in dataset:
        nz_flag = "mocks"
        cov_type = "cosmolike"
        # cov_type = "mocks"
        cosmology_template = "mice" + old_template
        cosmology_covariance = "mice"
        
        n_mocks = 1952

    theta_min, theta_max = GetThetaLimits(dataset=dataset_orig, nz_flag=nz_flag, dynamical_theta_limits=dynamical_theta_limits, code_path=code_path).get_theta_limits()
    
    mock_id_list = ["mean"] + list(range(n_mocks))
    mock_id_detected_list[dataset] = []
        
    fit_results[dataset] = {}
    
    for mock_id in mock_id_list:
        # 1. Arguments
        class Args:
            def __init__(self):
                self.include_wiggles = "y"
                self.dataset = dataset_orig
                self.weight_type = 1 # it will not be used...
                self.mock_id = mock_id
                self.nz_flag = nz_flag
                self.cov_type = cov_type
                self.cosmology_template = cosmology_template
                self.cosmology_covariance = cosmology_covariance
                self.delta_theta = 0.2
                self.theta_min = theta_min
                self.theta_max = theta_max
                self.pow_broadband = pow_broadband
                self.bins_removed = bins_removed
                self.diag_only = "n"
                self.remove_crosscov = "n"
                self.alpha_min = 0.8
                self.alpha_max = 1.2
                self.save_path = save_path
        args = Args()
        args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
        
        # 2. BAO fit initializer. This basically creates the path to load the results
        baofit_initializer = BAOFitInitializer(
            include_wiggles=args.include_wiggles,
            dataset=args.dataset,
            weight_type=args.weight_type,
            mock_id=args.mock_id,
            nz_flag=args.nz_flag,
            cov_type=args.cov_type,
            cosmology_template=args.cosmology_template,
            cosmology_covariance=args.cosmology_covariance,
            delta_theta=args.delta_theta,
            theta_min=args.theta_min,
            theta_max=args.theta_max,
            pow_broadband=args.pow_broadband,
            bins_removed=args.bins_removed,
            alpha_min=args.alpha_min,
            alpha_max=args.alpha_max,
            verbose=False,
            save_path=args.save_path,
        )
    
        try:
            results = np.loadtxt(os.path.join(baofit_initializer.path_baofit, "fit_results.txt"))
            if not os.path.exists(os.path.join(baofit_initializer.path_baofit, "wtheta_data_bestfit.txt")):
                print(f"Mock number {mock_id} has a non-detection! See {baofit_initializer.path_baofit}")
                # print(f"Mock number {mock_id} has a non-detection!")
            else:
                fit_results[dataset][mock_id] = results
                if mock_id != "mean":
                    mock_id_detected_list[dataset].append(mock_id)
                
        except:
            break
            
    print(f"Last mock available for the {dataset} data set: {mock_id}\n")


In [ ]:
# for dataset in dataset_mocks_list:

#     data = []
#     for key, results in fit_results[dataset].items():
#         mock_id = key
#         alpha, sigma_alpha, chi2, dof = results
#         data.append([mock_id, alpha, sigma_alpha, chi2, int(dof)])
    
#     df = pd.DataFrame(data, columns=["Mock id", r"$\alpha$", r"$\sigma_\alpha$", r"$\chi^2$", "dof"])
    
#     df[r"$\alpha$"] = df[r"$\alpha$"].map(lambda x: f"{x:.3f}")
#     df[r"$\sigma_\alpha$"] = df[r"$\sigma_\alpha$"].map(lambda x: f"{x:.3f}")
#     df[r"$\chi^2$"] = df[r"$\chi^2$"].map(lambda x: f"{x:.1f}")
#     df["dof"] = df["dof"].astype(int)
    
#     latex_table = df.to_latex(index=False, escape=False, column_format="c|cccc")

#     # print(f"LaTeX table for the {dataset} data set:\n")
#     # print(latex_table)


In [ ]:
summary_data = []

for dataset in dataset_mocks_list:
    alpha_mocks = []
    sigma_alpha_mocks = []
    chi2s = []
    dofs = []

    for mock_id, results in fit_results[dataset].items():
        alpha, sigma_alpha, chi2, dof = results
        if mock_id == "mean":
            alpha_mean_mocks = alpha
            sigma_alpha_mean_mocks = sigma_alpha
            chi2_mean_mocks = chi2
            dof_mean_mocks = dof
        else:
            alpha_mocks.append(alpha)
            sigma_alpha_mocks.append(sigma_alpha)
            chi2s.append(chi2)
            dofs.append(dof)

    alpha_mocks = np.array(alpha_mocks)
    sigma_alpha_mocks = np.array(sigma_alpha_mocks)
    chi2s = np.array(chi2s)
    dofs = np.array(dofs)

    alpha_mean = alpha_mocks.mean()
    sigma_std = alpha_mocks.std()

    upper, lower = np.percentile(alpha_mocks, [84.075, 15.825], method="linear")
    sigma_68 = (upper - lower) / 2

    mean_sigma_alpha = sigma_alpha_mocks.mean()

    frac_enc = np.mean(
        (alpha_mocks > alpha_mean - mean_sigma_alpha) &
        (alpha_mocks < alpha_mean + mean_sigma_alpha)
    ) * 100

    d_norm = (alpha_mocks - alpha_mean) / sigma_alpha_mocks
    dnorm_mean = d_norm.mean()
    dnorm_std = d_norm.std()

    mean_chi2 = chi2s.mean()
    mean_dof = dofs.mean() # they should all be the same...
    chi2_over_dof_str = f"{mean_chi2:.1f}/{int(round(mean_dof))}"

    alpha_mean_pm_sigma = f"{alpha_mean_mocks:.4f} ± {sigma_alpha_mean_mocks:.4f}"
    chi2_over_dof_str_mean = f"{chi2_mean_mocks:.1f}/{int(round(dof_mean_mocks))}"

    summary_data.append([
        dataset,
        alpha_mean,
        sigma_std,
        sigma_68,
        mean_sigma_alpha,
        frac_enc,
        dnorm_mean,
        dnorm_std,
        chi2_over_dof_str,
        alpha_mean_pm_sigma,
        chi2_over_dof_str_mean
    ])

summary_df = pd.DataFrame(summary_data, columns=[
    "Dataset",
    r"$\langle\alpha\rangle$",
    r"$\sigma_{\rm std}$",
    r"$\sigma_{68}$",
    r"$\langle\sigma_\alpha\rangle$",
    r"fraction encl.$\langle\alpha\rangle$ [\%]",
    r"$\langle d_{\rm norm}\rangle$",
    r"$\sigma_{d_{\rm norm}}$",
    r"$\langle\chi^2\rangle/$dof",
    "Mean of mocks",
    r"$\chi^2_{\rm mean\ mocks}/$dof",
])

# Format columns
for col in [
    r"$\langle\alpha\rangle$",
    r"$\sigma_{\rm std}$",
    r"$\sigma_{68}$",
    r"$\langle\sigma_\alpha\rangle$"
]:
    summary_df[col] = summary_df[col].map(lambda x: f"{float(x):.4f}")

summary_df[r"fraction encl.$\langle\alpha\rangle$ [\%]"] = summary_df[
    r"fraction encl.$\langle\alpha\rangle$ [\%]"].map(lambda x: f"{float(x):.1f}")

summary_df[r"$\langle d_{\rm norm}\rangle$"] = summary_df[
    r"$\langle d_{\rm norm}\rangle$"].map(lambda x: f"{float(x):.4f}")

summary_df[r"$\sigma_{d_{\rm norm}}$"] = summary_df[
    r"$\sigma_{d_{\rm norm}}$"].map(lambda x: f"{float(x):.4f}")

# Output LaTeX table
latex_table = summary_df.to_latex(index=False, escape=False, column_format="l|" + "c" * (len(summary_df.columns) - 1))
print(latex_table)


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"
import scipy

figsize = (18, 5)

for dataset in dataset_mocks_list:

    # Compute means
    mean_alpha = np.mean([fit_results[dataset][mock_id][0] for mock_id in mock_id_detected_list[dataset]])
    std_alpha = np.std([fit_results[dataset][mock_id][0] for mock_id in mock_id_detected_list[dataset]])
    mean_sigma_alpha = np.mean([fit_results[dataset][mock_id][1] for mock_id in mock_id_detected_list[dataset]])
    mean_chi2 = np.mean([fit_results[dataset][mock_id][2] for mock_id in mock_id_detected_list[dataset]])
    
    # Create 1x3 subplot grid
    fig, axs = plt.subplots(1, 3, figsize=figsize)

    fig.suptitle(fr"\texttt{{{dataset}}}", fontsize=20)
    
    # Histogram for alpha
    axs[0].hist([fit_results[dataset][mock_id][0] for mock_id in mock_id_detected_list[dataset]], 
                color='royalblue', edgecolor='black', alpha=0.7)
    axs[0].axvline(mean_alpha, color='black', linestyle="--", linewidth=2, label=fr'$\langle\alpha\rangle$ = {mean_alpha:.4f}')
    axs[0].set_xlabel(r'$\alpha$', fontsize=18)
    axs[0].tick_params(axis='both', labelsize=14)
    axs[0].legend(fontsize=14, loc='upper right')
    # axs[0].text(0.95, 0.85, fr'$\sigma_{{\rm std}}$ = {std_alpha:.4f}', ha='right', va='top', transform=axs[0].transAxes, fontsize=14)
    
    # Histogram for sigma_alpha
    axs[1].hist([fit_results[dataset][mock_id][1] for mock_id in mock_id_detected_list[dataset]], 
                color='darkorange', edgecolor='black', alpha=0.7)
    axs[1].axvline(mean_sigma_alpha, color='black', linestyle="--", linewidth=2, label=fr'$\langle\sigma_\alpha\rangle$ = {mean_sigma_alpha:.4f}')
    axs[1].set_xlabel(r'$\sigma_{\alpha}$', fontsize=18)
    axs[1].tick_params(axis='both', labelsize=14)
    axs[1].legend(fontsize=14)
    
    # Histogram for chi^2
    axs[2].hist([fit_results[dataset][mock_id][2] for mock_id in mock_id_detected_list[dataset]], density=True, 
                color='forestgreen', edgecolor='black', alpha=0.7)
    axs[2].axvline(mean_chi2, color='black', linestyle="--", linewidth=2, label=fr'$\langle\chi^2\rangle$ = {mean_chi2:.1f}')
    axs[2].set_xlabel(r'$\chi^2$', fontsize=18)
    axs[2].tick_params(axis='both', labelsize=14)
    
    # Overlay chi-squared distribution on the chi2 histogram
    dof = fit_results[dataset][mock_id_detected_list[dataset][0]][3]
    x = np.linspace(0, dof * 2, 500)
    pdf = scipy.stats.chi2.pdf(x, dof)
    axs[2].plot(x, pdf, label=fr'$\chi^2$ dist. (dof = {int(dof)})', color='darkgreen')
    axs[2].legend(loc='upper right', fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"plots/analysis_{dataset}.png")
    

In [ ]:
# Identify rows
full_row = summary_df.iloc[0]  # Full LRG
sub_rows = summary_df.iloc[1:]  # LRG1, LRG2, LRG3

# Extract arrays
alphas = sub_rows[r"$\langle\alpha\rangle$"].astype(float).values
sigmas = sub_rows[r"$\langle\sigma_\alpha\rangle$"].astype(float).values

# Compute weights (inverse variance)
weights = 1.0 / (sigmas**2)

# Combined alpha and sigma
alpha_comb = np.sum(alphas * weights) / np.sum(weights)
sigma_comb = np.sqrt(1.0 / np.sum(weights))

# Extract full LRG values
full_alpha = float(full_row[r"$\langle\alpha\rangle$"])
full_sigma = float(full_row[r"$\langle\sigma_\alpha\rangle$"])

# Print comparison
print("=== Comparison ===")
print(f"Full LRG:    alpha = {full_alpha:.4f}, sigma_alpha = {full_sigma:.4f}")
print(f"LRG1+LRG2+LRG3:    alpha = {alpha_comb:.4f}, sigma_alpha = {sigma_comb:.4f}")


# **This part is only for my DES stuff**

In [ ]:
# Load the results from the data

nz_flag = "fid"

fit_results_data = {}

for dataset in dataset_mocks_list:
    dataset = dataset.replace('_COLA', '')
    
    theta_min, theta_max = GetThetaLimits(dataset=dataset, nz_flag=nz_flag, dynamical_theta_limits=dynamical_theta_limits).get_theta_limits()
    
    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = "y"
            self.dataset = dataset
            self.weight_type = 1
            self.mock_id = "mean" # it will not be used...
            self.nz_flag = nz_flag
            self.cov_type = "cosmolike"
            self.cosmology_template = "planck" + old_template
            self.cosmology_covariance = "planck"
            self.delta_theta = 0.2
            self.theta_min = theta_min
            self.theta_max = theta_max
            self.pow_broadband = [-2, -1, 0]
            self.bins_removed = []
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.save_path = save_path
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    # 2. BAO fit initializer. This basically creates the path to load the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        pow_broadband=args.pow_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=False,
        save_path=args.save_path,
    )

    fit_results_data[dataset] = np.loadtxt(os.path.join(baofit_initializer.path_baofit, "fit_results.txt"))


In [ ]:
dataset_fid = "DESY6_COLA"
figsize = (5, 4)

for dataset in dataset_mocks_list[1:]:

    mock_id_detected_list_common = sorted(set(mock_id_detected_list[dataset_fid]) & set(mock_id_detected_list[dataset]))

    alpha_mocks_fid = np.array([fit_results[dataset_fid][mock_id][0] for mock_id in mock_id_detected_list_common])
    alpha_mocks_dataset = np.array([fit_results[dataset][mock_id][0] for mock_id in mock_id_detected_list_common])
    delta_alpha_mocks = alpha_mocks_dataset - alpha_mocks_fid

    alpha_data_fid = fit_results_data[dataset_fid.replace('_COLA', '')][0]
    alpha_data_dataset = fit_results_data[dataset.replace('_COLA', '')][0]
    delta_alpha_data = alpha_data_dataset - alpha_data_fid

    fig, ax = plt.subplots(1, 1, figsize=figsize)
    # plt.title(fr"\texttt{{{dataset_fid.replace('_COLA', '')}}} vs. \texttt{{{dataset.replace('_COLA', '')}}}", fontsize=20)
    plt.title(fr"\texttt{{{dataset.replace('_COLA', '').replace('DESY6_', '')}}}", fontsize=20)

    plt.hist(delta_alpha_mocks, edgecolor='black', alpha=0.7, label="COLA mocks", color="royalblue")
    plt.axvline(delta_alpha_data, color="violet", linewidth=3, label="Data")

    ax.set_xlabel(r'$\alpha - \alpha^{\mathrm{fid}}$', fontsize=18)
    ax.tick_params(axis='both', labelsize=14)

    plt.legend(loc="best", fontsize=15)
    
    plt.tight_layout()
    plt.savefig(f"plots/diff_alpha_{dataset.replace('_COLA', '')}_vs_{dataset_fid.replace('_COLA', '')}.png")

    # pct = np.sum(delta_alpha_mocks > delta_alpha_data) / len(mock_id_detected_list_common) * 100
    # print(f"{pct:.2f}% of the mocks have larger delta alpha")
    
    # corr = np.corrcoef(alpha_mocks_fid, alpha_mocks_dataset)[0, 1]
    # print(f"Correlation between {dataset_fid} and {dataset}: {corr:.2f}")

    print(f"Data delta alpha: {delta_alpha_data*100:.2f}")
    for lower_limit, upper_limit in zip([5, 0.5], [95, 99.5]):
        lower, upper = np.percentile(delta_alpha_mocks, [lower_limit, upper_limit])
        print(f"{int(upper_limit - lower_limit)}% interval for mocks: [{lower*100:.2f}, {upper*100:.2f}]")
    percentile = scipy.stats.percentileofscore(delta_alpha_mocks, delta_alpha_data, kind='rank')
    print(f"Percentile of data: {percentile:.2f}%\n")


In [ ]:
dataset_fid = "DESY6_COLA"
figsize = (5, 4)

for dataset in dataset_mocks_list[1:]:
    
    mock_id_detected_list_common = sorted(set(mock_id_detected_list[dataset_fid]) & set(mock_id_detected_list[dataset]))

    sigma_alpha_mocks_fid = np.array([fit_results[dataset_fid][mock_id][1] for mock_id in mock_id_detected_list_common])
    sigma_alpha_mocks_dataset = np.array([fit_results[dataset][mock_id][1] for mock_id in mock_id_detected_list_common])
    ratio_sigma_alpha_mocks = sigma_alpha_mocks_dataset / sigma_alpha_mocks_fid

    sigma_alpha_data_fid = fit_results_data[dataset_fid.replace('_COLA', '')][1]
    sigma_alpha_data_dataset = fit_results_data[dataset.replace('_COLA', '')][1]
    ratio_sigma_alpha_data = sigma_alpha_data_dataset / sigma_alpha_data_fid

    fig, ax = plt.subplots(1, 1, figsize=figsize)
    # plt.title(fr"\texttt{{{dataset_fid.replace('_COLA', '')}}} vs. \texttt{{{dataset.replace('_COLA', '')}}}", fontsize=20)
    plt.title(fr"\texttt{{{dataset.replace('_COLA', '').replace('DESY6_', '')}}}", fontsize=20)

    plt.hist(ratio_sigma_alpha_mocks, edgecolor='black', alpha=0.7, label="COLA mocks", color="darkorange")
    plt.axvline(ratio_sigma_alpha_data, color="violet", linewidth=3, label="Data")

    ax.set_xlabel(r'$\sigma_\alpha / \sigma_\alpha^{\mathrm{fid}}$', fontsize=18)
    ax.tick_params(axis='both', labelsize=14)

    plt.legend(loc="best", fontsize=15)
    
    plt.tight_layout()
    plt.savefig(f"plots/ratio_sigma_{dataset.replace('_COLA', '')}_vs_{dataset_fid.replace('_COLA', '')}.png")

    pct = np.sum(ratio_sigma_alpha_mocks > ratio_sigma_alpha_data) / len(mock_id_detected_list_common) * 100
    print(f"{pct:.2f}% of the mocks have a higher sigma_alpha ratio than data")

    print(f"Data delta alpha: {ratio_sigma_alpha_data:.2f}")
    for lower_limit, upper_limit in zip([5, 0.5], [95, 99.5]):
        lower, upper = np.percentile(ratio_sigma_alpha_mocks, [lower_limit, upper_limit])
        print(f"{int(upper_limit - lower_limit)}% interval for mocks: [{lower:.2f}, {upper:.2f}]")
    percentile = scipy.stats.percentileofscore(ratio_sigma_alpha_mocks, ratio_sigma_alpha_data, kind='rank')
    print(f"Percentile of data: {percentile:.2f}%\n")
    

In [ ]:
figsize = (5, 4)

for dataset in dataset_mocks_list:

    fig, ax = plt.subplots(1, 1, figsize=figsize)

    plt.title(fr"\texttt{{{dataset.replace('_COLA', '')}}}", fontsize=20)

    plt.plot([fit_results[dataset][mock_id][0] for mock_id in mock_id_detected_list[dataset]], [fit_results[dataset][mock_id][1] for mock_id in mock_id_detected_list[dataset]], ".")

    plt.plot(fit_results_data[dataset.replace("_COLA", "")][0], fit_results_data[dataset.replace("_COLA", "")][1], "d")

    ax.set_xlabel(r'$\alpha$', fontsize=18)
    ax.set_ylabel(r'$\sigma_\alpha$', fontsize=18)
    ax.tick_params(axis='both', labelsize=14)
    
    plt.tight_layout()
    plt.show()
    

In [ ]:
dataset = 'DESY6_COLA'

sigma_alpha_mean_mocks = {}

for dataset in dataset_mocks_list:

    sigma_alpha_mean_mocks[dataset] = np.array([fit_results[dataset][mock_id][1] for mock_id in mock_id_detected_list[dataset]]).mean()

print(sigma_alpha_mean_mocks['DESY6_COLA'])
print(1 / np.sqrt(1 / sigma_alpha_mean_mocks['DESY6_COLA_DR1tiles_noDESI']**2 + 1 / sigma_alpha_mean_mocks['DESY6_COLA_DR1tiles_DESIonly']**2))
print(1 / np.sqrt(1 / sigma_alpha_mean_mocks['DESY6_COLA_dec_below-23.5']**2 + 1 / sigma_alpha_mean_mocks['DESY6_COLA_dec_above-23.5']**2))


In [ ]:
dataset = 'DESY6_COLA'

sigma_alpha_mean_mocks = {}

for dataset in dataset_mocks_list:

    sigma_alpha_mean_mocks[dataset] = fit_results[dataset][15][1]

print(sigma_alpha_mean_mocks['DESY6_COLA'])
print(1 / np.sqrt(1 / sigma_alpha_mean_mocks['DESY6_COLA_DR1tiles_noDESI']**2 + 1 / sigma_alpha_mean_mocks['DESY6_COLA_DR1tiles_DESIonly']**2))
print(1 / np.sqrt(1 / sigma_alpha_mean_mocks['DESY6_COLA_dec_below-23.5']**2 + 1 / sigma_alpha_mean_mocks['DESY6_COLA_dec_above-23.5']**2))
